# Data Analysis

## Dependencies

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

## Import Data

In [9]:
account_data = pd.read_csv("../../assets/clean/account.csv")
card_data = pd.read_csv("../../assets/clean/card_dev.csv")
client_data = pd.read_csv("../../assets/clean/client.csv")
disp_data = pd.read_csv("../../assets/clean/disp.csv", dtype={"disp_id": int, "client_id": int, "account_id": int, "type": str})
district_data = pd.read_csv("../../assets/clean/district.csv")
loan_data = pd.read_csv("../../assets/clean/loan_dev.csv")
transaction_data = pd.read_csv("../../assets/clean/trans_dev.csv", dtype=
    {"trans_id": int, "account_id": int, "date": str, "type": str, "operation": str, "amount": float, "balance": float, "k_symbol": str, "bank": str, "account": str}
)


## Data Exploratory Analysis

In [3]:
# Variable used to avoid re-running certain cells
DATA_IS_LOADED = False

### Change Pandas display settings

In [4]:
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

### Analyze the Size of each Dataset

In [5]:
print("=============================================\n")
print("Number of rows for each dataset: \n")
print(f"Account: {len(account_data)}")
print(f"Card: {len(card_data)}")
print(f"Client: {len(client_data)}")
print(f"Disposition: {len(disp_data)}")
print(f"District: {len(district_data)}")
print(f"Loan: {len(loan_data)}")
print(f"Transaction: {len(transaction_data)}")


Number of rows for each dataset: 

Account: 4500
Card: 177
Client: 5369
Disposition: 5369
District: 77
Loan: 328
Transaction: 396685


### Get Missing Values

In [10]:
print("=============================================\n")
print("Missing values for each dataset: \n")
print(f"Account: \n{account_data.isnull().sum()} \n")
print(f"Card: \n{card_data.isnull().sum()} \n")
print(f"Client: \n{client_data.isnull().sum()} \n")
print(f"Disposition: \n{disp_data.isnull().sum()} \n")
print(f"District: \n{district_data.isnull().sum()} \n")
print(f"Loan: \n{loan_data.isnull().sum()} \n")
print(f"Transaction: \n{transaction_data.isnull().sum()} \n")

# TODO: Confirm if missing values are all being tracked. Values such as '?' exist in the original dataset


Missing values for each dataset: 

Account: 
account_id     0
district_id    0
frequency      0
date           0
dtype: int64 

Card: 
card_id    0
disp_id    0
type       0
issued     0
dtype: int64 

Client: 
client_id       0
birth_number    0
district_id     0
gender          0
dtype: int64 

Disposition: 
disp_id       0
client_id     0
account_id    0
type          0
dtype: int64 

District: 
code                 0
name                 0
region               0
num_inhabitants      0
municip499           0
municip500_1999      0
municip2000_9999     0
municip10000         0
no. of cities        0
urban_ratio          0
avg_salary           0
unemp_rate95         1
unemp_rate96         0
num_entrepreneurs    0
num_crimes95         1
num_crimes96         0
dtype: int64 

Loan: 
loan_id       0
account_id    0
date          0
amount        0
duration      0
payments      0
paid          0
dtype: int64 

Transaction: 
trans_id           0
account_id         0
date               0
typ

#### Replace Missing Values

To have a consistent dataset, the columns containing missing values should be handled. In the `transaction` table, the `k_symbol`, `bank`, `account` and `operation` columns have missing values. The first three of those are categorical columns, so the missing values are replaced below with an `unkown` value. As for the `operation` column, it's explained below that it is strongly correlated with the `type` column, so it is dropped.

Regarding the `district` table, there is one row with missing values on the columns `unemp_rate95` and `num_crimes95`. In these cases, since they are ratios, we can replace them with the mean of the column.

In [11]:
district_data.fillna(district_data.mean(), inplace=True)
print(district_data.mean())

code                     39.000000
num_inhabitants      133884.896104
municip499               48.623377
municip500_1999          24.324675
municip2000_9999          6.272727
municip10000              1.727273
no. of cities             6.259740
urban_ratio              63.035065
avg_salary             9031.675325
unemp_rate95              3.119342
unemp_rate96              3.787013
num_entrepreneurs       116.129870
num_crimes95           4850.315789
num_crimes96           5030.831169
dtype: float64


/tmp/ipykernel_6566/2139339797.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  district_data.fillna(district_data.mean(), inplace=True)
/tmp/ipykernel_6566/2139339797.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(district_data.mean())


### Analyze Loans

In [ ]:
print("=============================================\n")
paid_loans = loan_data[loan_data["paid"] == 1]
unpaid_loans = loan_data[loan_data["paid"] == 0]
paid_loans_percentage = round((len(paid_loans)/len(loan_data)) * 100, 2)
print(f"{paid_loans_percentage}% of loans are paid.")

Approx 86% of loans have been paid (positive result). This means
means that accuracy isn't a good metric to optimize for

In [ ]:
plt.figure()
plt.bar([0, 1], loan_data["paid"].value_counts(), tick_label=["Paid", "Unpaid"])
# plt.ylim(0, 300)
plt.title("Loans paid vs unpaid")
plt.xlabel("Paid")
plt.ylabel("Frequency")
plt.show()

### Investigate attributes with a large quantity of nulls

In [ ]:
# TOOD: SPLIT THIS INTO DIFFERENT CELLS

def parse_k_symbol(k_symb):
    if isinstance(k_symb, float):
        return 'none'
    elif k_symb == " ":
        return 'none'
    else:
        return k_symb

print("=============================================\n")
print("[Operation]:")
print(transaction_data["operation"].value_counts())
operationNullsPercentage = round(len(transaction_data[transaction_data["operation"].isnull()]) / len(transaction_data["operation"]) * 100, 2)
print(f"Nulls (%): {operationNullsPercentage}%")
''' The operation attribute is categorical and doesn't reveal an inherited order. It can be encoded with 3 attributes
using binary encoding. We will fill the 'nulls' with "unknown"
'''

transaction_data['operation'].fillna("unknown", inplace=True)
# print(transaction_data["operation"].value_counts())

print("\n[k_symbol]:")
# print(transaction_data["k_symbol"].value_counts())
''' The k_symbol attribute is categorical and doesn't reveal an inherited order. It can be encoded with 3 attributes
using binary encoding. We will replace the " " with "none"
'''

transaction_data["k_symbol"] = transaction_data["k_symbol"].apply(parse_k_symbol)
print(transaction_data["k_symbol"].value_counts())
k_symbol_none_percentage = round(len(transaction_data[transaction_data["k_symbol"] == "none"]) / len(transaction_data["k_symbol"]) * 100, 2)
print(f"Nulls (%): {k_symbol_none_percentage}%")


print("\n[bank]:")
#print(transaction_data["bank"].value_counts())
print("Number of unknown banks:", len(transaction_data[transaction_data['bank'].isnull()]))

print("\n[account]:")
# print(transaction_data["account"].value_counts())
print("Number of unknown partners:", len(transaction_data[transaction_data['account'].isnull()]) + len(transaction_data[transaction_data['account'] == "0"]))

rows = transaction_data[transaction_data["bank"].isnull() & (~transaction_data["account"].isnull()) & (transaction_data["account"] != "0")]
print(f"Number of rows where bank is empty and the account isn't: {len(rows)}")
'''This proves that these 2 feature are highly correlated. Although these 2 attributes aren't likely to contribute anything to our prediction, we can create an
"unknown" bank and set all unknown account attributes to 0.'''

transaction_data["account"].fillna(0, inplace=True)
transaction_data["bank"].fillna("unknown", inplace=True)

In [ ]:
# Auxiliary method to join 2 datasets
def join(df1, df2, key1, key2, suff, t="inner"):
    return df1.merge(df2, left_on=key1, right_on=key2, how=t, suffixes=suff)

## Dimensions of Data Quality

**Data quality dimension 6** - *Timeliness*
- The data in the dataset is outdated. Consequently, it does not achieve *timeliness*

**Data quality dimension 1** - *Completeness*
- Some mandatory fields were missing, hence the data cannot be considered *complete*.

**Data quality dimension 5** - *Integrity*
- Altough the objective of our work is to predict whether a loan will end successfuly, most of the accounts do not have associated loan requests - There are orphaned records. This means that the data lacks *Integrity*

**Data quality dimension 2** - *Consistency*
- A prime example of the lack of consistency is the overlapping info between the columns *type* and *operation* of the Transactions table. This info can be seen in the [*Join Transactions* section](#transactions_type_operation_comp)

**Data quality dimension 3** - *Conformity*
- Dates follow the format: "yy-mm-dd" in general. However, the client's birth date does not follow this format, since it adds 50 to the month part of the number. Thus, the data is not *conformant* 

### Join Data

It is needed to join all datasets into a single Dataframe

#### Join Account with Disposition

Let's first analyze the Disposition dataset:

In [ ]:
print("Number of clients per disposition type:")
print(disp_data["type"].value_counts())

sb.displot(disp_data, x="type", hue="type")
plt.show()

All Accounts have at least 1 Disposition. There are more Dispositions than Accounts, since some clients are **owners** while others are **disponent owners** 

We will create an attribute on the Account table that reflects whether the account is co-owned. Note that the DISPONENTs will be dropped from the joined table. The type attribute can be dropped as well since all dispositions are Owners.

In [ ]:
df = join(account_data, disp_data, "account_id", "account_id", ["", "_disp"])
df.rename(columns={"date": "a_date"}, inplace=True)

# Count Groups
owner_count = df["account_id"].value_counts()
df["is_co-owned"] = df.apply(lambda row: 1 if owner_count[row["account_id"]] > 1 else 0, axis='columns')

# Cleanup
df.drop(df[df["type"] == "DISPONENT"].index, inplace=True)
df.drop(columns=["type"], inplace=True)

TODO: The column "disp_id" might also be useless since it's a 1-1 relation with the account now. But we
 will drop it after proving the correlation between the 2 attributes.'

### Join Clients

In [ ]:
if not DATA_IS_LOADED:
    df = join(df, client_data, "client_id", "client_id", ["", "_client"], t="left")
    df.drop(['client_id'], axis='columns', inplace=True, errors='ignore')

#### Join Districts

Both the *Account* and the *Client* have a foreign key to the *District* table
We will for now add *District* info about both

In [ ]:
if not DATA_IS_LOADED:
    # Add Client's District Data -> Change to MD
    df = join(df, district_data, "district_id_client", "code", ["", "_district"], t="inner")
    df.drop(['district_id_client'], axis='columns', inplace=True)

    # Add Accounts's District Data -> Change to MD
    df = join(df, district_data, "district_id", "code", ["_aDistrict", "_cDistrict"], t="inner")
    df.drop(['district_id'], axis='columns', inplace=True)
    
# df


Note that the district information regarding the *Account* contains an `a` prefix, while the *Client* info contains a `c` prefix.

Some of the Columns such as: *code_aDistrict* and *name_aDistrict* seem to be ambiguous. However, we will first confirm their correlation before removing them.

The next 3 tables have training and testing versions. Since we will do the data analysis of the training version, we'll ignore the other one for now

### Join Loan

In [ ]:
if not DATA_IS_LOADED:
    # Merge to the right, since accounts that don't have loans are not relevant
    df = join(df, loan_data, "account_id", "account_id", ["", "_loan"], t="right")
    df.drop(['loan_id'], axis='columns', inplace=True)
    df.rename(columns={"date": "loan_date"}, inplace=True)
df

### Join Card

Almost no *Client* in the dataset has a *Card*. This is rather problematic, since it creates a lot of missing values or lost of information.

The *Card* dataset contains only 2 useful columns:
- Card type -- can be ordered from worst to best
- Issuance date

The type of Card helps us rank clients. A *Client* with a gold card is better than a client with a junior card. On the other hand, the **issuance date** allows us to consider only cards that have been issued before the loan, as information after the loan is irrelevant for this prediction task. TODO: THIS LAST PART IS NOT DONE

The Card **type** is a categorical field:
- *junior* -- underage people (lowest tier), usually it isn't possible to withdraw money from them
- *classic*
- *gold* -- highest tier

**Since the field is ordered in tiers, in the cleaning process we should convert these values to a numerical scale where:**
- 0 -> no card
- 1 -> junior
- 2 -> classic
- 3 -> gold

In [ ]:
if not DATA_IS_LOADED:
    card_disp = join(card_data, disp_data, "disp_id", "disp_id", ["_card", "_disp"])
    card_disp = card_disp.groupby([card_disp.account_id, card_disp.type_card]).size().unstack(fill_value=0)
card_disp

In [ ]:
if not DATA_IS_LOADED:
    df = join(df, card_data, "disp_id", "disp_id", ["", "_card"], t="left")
    df.fillna("none", inplace=True)
    df.drop(['card_id', 'disp_id'], axis='columns', inplace=True)
    df.rename(columns={"type": "card_type", "issued": "card_issued"}, inplace=True)

df


There are very few clients with cards. Every *Client* with a card has been granted a loan

In [ ]:
df["card_type"].value_counts()

In [ ]:
df[df["card_type"] != "none"]["paid"].value_counts()

### Join Transactions

We need to solve an issue with the *Transactions* table -- There are many transactions for some accounts. So, joining them would create multiple rows for each loan.

We need to find a way to extract the useful information from this table to merge it into the dataset.

Note that in this table the *account_id* columns refers to the **target account** of the operation, while the *account* refers to the source 

This table contains multiple categorical attributes. Before working with it, we should explore in more detail what they represent

In [ ]:
transaction_data['type'].value_counts(dropna=False)

In [ ]:
transaction_data['operation'].value_counts(dropna=False)

<a id="transactions_type_operation_comp"></a>

The fields: *type* and *operation* seem to transmit identical information, where the *operation* goes into more detail about the type of transaction. Thus, after investigating the dataset we found the following mapping:

| type               | operation                                                              |
|--------------------|------------------------------------------------------------------------|
| credit             | unknown; collection from another bank, credit in cash                  |
| withdrawal         | credit card withdrawal; remittance to another bank; withdrawal in cash |
| withdrawal in cash | withdrawal in cash                                                     |

Consequently, this ambiguity will be taken into account towards the building of the final dataset.

In [ ]:
transaction_data['k_symbol'].value_counts(dropna=False)

In [ ]:
def filter_missing_accounts():
    account_ids = df["account_id"].unique()
    return transaction_data.copy().loc[transaction_data["account_id"].isin(account_ids)]

if not DATA_IS_LOADED:
    active_transactions = filter_missing_accounts()

active_transactions.shape

Remove *Transactions* that happened after the respective loan decision 

In [ ]:
RUN_COUNT_LATE_TRANSACTIONS = False # Bolean to skip this instruction if desired

def count_late_transactions():
    counter = 0
    for id, row in active_transactions.iterrows():
        transaction_date = row["date"]
        acc_id = row["account_id"]
        for _, loan_row in df[df["account_id"] == acc_id].iterrows():
            loan_date = loan_row["loan_date"]
            if transaction_date > loan_date:
                counter += 1
    
    return counter

if RUN_COUNT_LATE_TRANSACTIONS:
    if not DATA_IS_LOADED:
        late_transactions_count = count_late_transactions()

    print(f"Number of late transactions: {late_transactions_count}")

All *Transactions* happened before the loan decision, so none were dropped

To simplify the understanding of transactions:
- Withdrawals will have a negative *balance*
- Credit will have a positive *balance*

In [ ]:
def convert_balance(transactions):
    def convert_bal_inner(row):
        amount = abs(row['amount'])
        return amount if row['type'] == 'credit' else -amount
    
    transactions['amount'] = transactions.apply(convert_bal_inner, axis='columns')
    return transactions

if not DATA_IS_LOADED:
    active_transactions = convert_balance(active_transactions)
active_transactions

Regarding the *bank* and *account* columns, most entries are empty (83%). So these columns were considered not useful.

In [ ]:
active_transactions[active_transactions['bank'] == "unknown"]["bank"].count() / active_transactions["bank"].count()

In [ ]:
if not DATA_IS_LOADED:
    active_transactions.drop(['bank', 'account'], axis='columns', inplace=True)

active_transactions

### Sanctions for Negative Balance
Knowing the number of times each *User* was sanctioned for **negative balance** might be useful to know which loans to reject

In [ ]:
def count_sanctions(df, active_transactions):
    grouped = active_transactions.groupby('account_id')

    sanction_df = pd.DataFrame(columns=['account_id', 'sanctions'])
    for acc_id, group in grouped:
        sanction_count = group[group['k_symbol'] == "sanction interest if negative balance"]["k_symbol"].count()
        sanction_df = pd.concat([sanction_df, pd.DataFrame({'account_id': acc_id, 'sanctions': sanction_count}, index=[0])], ignore_index=True)

    df["sanctions"] = sanction_df["sanctions"]
    df["sanctions"].fillna(0.0, inplace=True)
    return df

if not DATA_IS_LOADED:
    df = count_sanctions(df, active_transactions)

# df["sanctions"].value_counts()
df
    

### Mean Household Payments

Knowing how much a client pays for its house may be a useful metric to know its financial situation

In [ ]:
def household_mean(df, active_transactions):
    grouped = active_transactions.groupby("account_id")
    household_df = pd.DataFrame(columns=["account_id", "household_mean"])

    for acc_id, group in grouped:
        house_mean = abs(group[group["k_symbol"] == "household"]["amount"].mean())
        household_df = pd.concat([household_df, pd.DataFrame({'account_id': acc_id, 'household_mean': house_mean}, index=[0])], ignore_index=True)

    df["household_mean"] = household_df["household_mean"]
    df["household_mean"].fillna(0.0, inplace=True)
    return df

if not DATA_IS_LOADED:
    df = household_mean(df, active_transactions)

df["household_mean"].head()

### Mean Interest Payments
Knowing how much a client earns in **account interest** might be useful

In [ ]:
def interest_mean(df, active_transactions):
    grouped = active_transactions.groupby("account_id")
    interest_df = pd.DataFrame(columns=["account_id", "interest_mean"])

    for acc_id, group in grouped:
        interest_mean = abs(group[group["k_symbol"] == "interest credited"]["amount"].mean())
        interest_df = pd.concat([interest_df, pd.DataFrame({'account_id': acc_id, 'interest_mean': interest_mean}, index=[0])], ignore_index=True)

    df["interest_mean"] = interest_df["interest_mean"]
    df["interest_mean"].fillna(0.0, inplace=True)
    return df

if not DATA_IS_LOADED:
    df = interest_mean(df, active_transactions)

df["interest_mean"].head()

### Mean Balance in Time References

The mean balance in the account **1 month**, **1 semester**, and **1 year** before the loan request

In [ ]:
def balance_time_references(df, active_transactions):
    def to_datetime(df, col_name="date"):
        df[col_name + "_dt"] = pd.to_datetime(df[col_name], format="%Y-%m-%d")
    
    to_datetime(df, "loan_date")
    to_datetime(active_transactions)

    # Join current dataset with transactions so that we can group by account_id
    temp_df = join(df, active_transactions, "account_id", "account_id", suff=["", "_trans"])[["account_id", "trans_id", "date_dt", "balance", "loan_date_dt"]]

    # Calculate month's offset between each transaction and loan date
    temp_df['month_diff'] = (temp_df['loan_date_dt'] - temp_df['date_dt']) / np.timedelta64(1, 'M')

    # 
    def avg_balance(dataframe, balance_name):
        return dataframe.groupby('account_id').mean()['balance'].reset_index().rename(columns={'balance': balance_name})

    # Create 3 Tables for the average balance in an account in the previous month, semester and year (account_id, balance)
    prev_year = avg_balance(temp_df.loc[temp_df['month_diff'] <= 12], 'prev_year_balance') 
    prev_semester = avg_balance(temp_df.loc[temp_df['month_diff'] <= 6], 'prev_6m_balance')
    prev_month = avg_balance(temp_df.loc[temp_df['month_diff'] <= 1], 'prev_1m_balance')

    mergedDf = df.merge(prev_year, on='account_id', how='inner')
    mergedDf = mergedDf.merge(prev_semester, on='account_id', how='inner')
    mergedDf = mergedDf.merge(prev_month, on='account_id', how='inner')

    mergedDf.drop(['loan_date_dt'], axis='columns', inplace=True)

    return mergedDf

if not DATA_IS_LOADED:
    df = balance_time_references(df, active_transactions)

df
    

### Min, Max and Mean Balances

it's also relevant to store some statistics regarding the balance of the clients along time

In [ ]:
def client_balance(df, active_transactions):
    grouped = active_transactions.groupby("account_id")
    balance_df = pd.DataFrame(columns=["account_id", "balance_min", "balance_max", "balance_mean"])

    for acc_id, group in grouped:
        client_balance = group["balance"]
        balance_df = pd.concat([balance_df, pd.DataFrame({'account_id': acc_id, 'balance_min': client_balance.min(), 'balance_max': client_balance.max(), 
            'balance_mean': client_balance.mean()}, index=[0])], ignore_index=True)

    balance_df.fillna(0.0, inplace=True)
    df = join(df, balance_df, "account_id", "account_id", suff=['', ''])
    return df

if not DATA_IS_LOADED:
    df = client_balance(df, active_transactions)

df

### Average balance Influx
Knowing the **balance variation** of a client might prove to be an useful metric, as it portrays the tendencies of the account. Consequently, we will create 3 columns to store the average influx (money IN - money OUT) in the last 3, 6 and 12 months.


In [ ]:
def client_bal_influx(df, active_transactions):
    def to_datetime(df, col_name="date"):
        df[col_name + "_dt"] = pd.to_datetime(df[col_name], format="%Y-%m-%d")

    to_datetime(df, "loan_date")
    to_datetime(active_transactions)

    # Join current dataset with transactions so that we can group by account_id
    temp_df = join(df, active_transactions, "account_id", "account_id", suff=["", "_trans"])[["account_id", "date_dt", "loan_date_dt", "amount_trans"]]
    temp_df.rename(columns={'amount_trans': 'influx'}, inplace=True)

    # Calculate month's offset between each transaction and loan date
    temp_df['month_diff'] = (temp_df['loan_date_dt'] - temp_df['date_dt']) / np.timedelta64(1, 'M')

    # Calculate sum of influxes from the given Rows
    def balance_influx(dataframe, balance_name):
        return dataframe.groupby('account_id').sum(numeric_only=True)['influx'].reset_index().rename(columns={'influx': balance_name})

    # Create 3 Tables for the average influx in the last 3, 6 and 12 months
    prev_year = balance_influx(temp_df.loc[temp_df['month_diff'] <= 12], 'mean_year_influx')
    prev_year['mean_year_influx'] = prev_year['mean_year_influx'] / 12  # Average Influx per Month

    prev_6month = balance_influx(temp_df.loc[temp_df['month_diff'] <= 6], 'mean_6m_influx')
    prev_6month['mean_6m_influx'] = prev_6month['mean_6m_influx'] / 6  # Average Influx per Month

    prev_3month = balance_influx(temp_df.loc[temp_df['month_diff'] <= 3], 'mean_3m_influx')
    prev_3month['mean_3m_influx'] = prev_3month['mean_3m_influx'] / 3  # Average Influx per Month

    mergedDf = df.merge(prev_year, on='account_id', how='inner')
    mergedDf = mergedDf.merge(prev_6month, on='account_id', how='inner')
    mergedDf = mergedDf.merge(prev_3month, on='account_id', how='inner')

    mergedDf.drop(['loan_date_dt'], axis='columns', inplace=True)

    return mergedDf

if not DATA_IS_LOADED:
    df = client_bal_influx(df, active_transactions)

df

Now that we have extracted the desired information from all the tables, we can drop the account_id

In [ ]:
if not DATA_IS_LOADED:
     df.drop(['account_id'], axis='columns', inplace=True)

### Check Duplicate rows

In [ ]:
df.drop(df.drop_duplicates().index)

There are no duplicate rows

## Save the Joined Data

In [ ]:
from pathlib import Path

filepath = Path("../../assets/joined")
filepath.mkdir(parents=True, exist_ok=True)

df.to_csv("../../assets/joined/data.csv", index=False)

### Final Dataset:

In [ ]:
df

===================

In [ ]:
# Variable used to avoid re-running certain cells
DATA_IS_LOADED = True